Consigna:
def userdata(User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [77]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np

In [ ]:
''' Lectura de Archivos '''
# Se abren los 3 dataframes por que el data frame de la función necesita por lo menos un dato de cada uno

df_steam_games = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_steam_games.parquet"
df_steam_games = pd.read_parquet(df_steam_games )
df_steam_games

In [111]:
df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_user_reviews = pd.read_parquet(df_user_reviews)
df_user_reviews

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,2
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,2
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,2


In [110]:
df_users_items = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_users_items.parquet"
df_users_items = pd.read_parquet(df_users_items)
df_users_items 

,user_id,items_count,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5153204,76561198147832762,31,http://steamcommunity.com/profiles/76561198147...,346330,BrainBread 2,0.0,0.0
5153205,76561198147832762,31,http://steamcommunity.com/profiles/76561198147...,373330,All Is Dust,0.0,0.0
5153206,76561198147832762,31,http://steamcommunity.com/profiles/76561198147...,388490,One Way To Die: Steam Edition,3.0,3.0
5153207,SomeRandomNewAccount,8,http://steamcommunity.com/id/SomeRandomNewAccount,521570,You Have 10 Seconds 2,4.0,4.0


In [ ]:
# Inicio el dataframe de la función por user_items

df_userdata = df_users_items
df_userdata

In [85]:
df_userdata.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5062823 entries, 0 to 5153208
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   user_url          object 
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  float64
 6   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 309.0+ MB


In [86]:
# Cambio el tipo de laytime forever de float 63 a float 32 por espacio

df_userdata['playtime_forever'] = df_userdata['playtime_forever'].astype('float32')

In [ ]:
df_userdata.info()

In [88]:
# De este df solo necesito user_id, item_id, 'items_count', item_name y play time_forever, así que

columnas_borrar = [ 'user_url', 'playtime_2weeks'] 

df_userdata = df_userdata.drop(columnas_borrar,axis=1)

In [ ]:
df_userdata.head()

In [ ]:
# Reviso Stam_Games pues lo relacionare con el precio 
df_steam_games.head()

In [ ]:
# De steam_games solo necesito item_id y price asi que borro las demás columnas
# Item_id = app_name

columnas_borrar2 = ['publisher', 'genres', 'url', 'release_date', 'reviews_url', 'specs', 'id', "early_access", 'developer', 'release_year'] 

df_steam_games = df_steam_games .drop(columnas_borrar2, axis=1)

df_steam_games

In [ ]:
df_steam_games.head()

In [92]:
# Cambio nombre de columna app_name por "item_name" para que pueda hacer el match

df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})


In [ ]:
# reviso la columna price de steam-games

df_steam_games.info()

In [94]:
# Modifico de una vez la columna price para que se pueda sumar luego

df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games = df_steam_games.dropna(subset=['price'])

In [ ]:
df_steam_games.info()

In [ ]:
df_steam_games.info()

In [97]:
# Como no me dejó convertir Convierto de float 64 a float 32 por temas de espacio, me toca  crear arreglo y convertirlo luego
#así no me djejo:  df_steam_games['price'] .astype(np.float32)

arreglo = df_steam_games['price'] .to_numpy() 


In [98]:
# converto el arreglo en float 32

df_float32 = arreglo.astype('float32')

In [ ]:
df_float32

In [100]:
# Como ya tengo arreglo en float 32, vou a cambar la columna price por esta en steam_games

df_steam_games['price'] = df_float32


In [ ]:
df_steam_games.info()

In [102]:
# Relaciono steam_games con userdata a travez de item_name para aumentar la coloumna  precio al dataframe

df_userdata = pd.merge(df_userdata, df_steam_games[['item_name', 'price']], on='item_name', how='left')  


In [ ]:
df_userdata.info()

In [107]:
df_userdata.shape

(32746611, 6)

In [244]:
# retiro na y duplicados

df_userdata = df_userdata.drop_duplicates()


In [245]:
df_userdata = df_userdata.dropna

In [246]:
df_userdata.shape

AttributeError: 'function' object has no attribute 'shape'

In [221]:
df_userdata2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3060168 entries, 0 to 43096383
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
 4   price             string 
dtypes: float64(1), object(3), string(1)
memory usage: 140.1+ MB


In [ ]:
# Solo queda faltando la columna recommend de user reviews y hare la relación a travez de item_id}

df_userdata = pd.merge(df_userdata, df_user_reviews, on='item_id', how='inner')
df_userdata 